In [29]:
# importação de bilbiotecas
import pyspark
import boto3
from io import StringIO 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [3]:
# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
spark = SparkSession.builder \
      .master("local[2]") \
      .appName("AwariAula08Ex") \
      .getOrCreate() 

In [4]:
spark.stop()


conf = pyspark.SparkConf()

# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
conf.setMaster("local[2]") 
conf.set("spark.driver.host", "awari-jupyterlab") \
    .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
    .set("parquet.enable.summary-metadata", "false") \
    .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .set("spark.driver.port", "20020") \
    .set("spark.hadoop.fs.s3a.endpoint", 'awari-nginx:9000') \
    .set("spark.hadoop.fs.s3a.endpoint.region", 'sa-east-1') \
    .set("spark.hadoop.fs.s3a.access.key", 'rwGPDEWtGtCIQnKe') \
    .set("spark.hadoop.fs.s3a.secret.key", 'dQWHX0cNzHFM83UZOiKlsPdpNK4PRMtm') \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .set("spark.hadoop.com.amazonaws.services.s3.enableV2", "true") \
    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")

conf.setAppName('AwariAula08Ex')
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession(sc)

In [30]:
# conexao com MinIo
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='rwGPDEWtGtCIQnKe',
    aws_secret_access_key='dQWHX0cNzHFM83UZOiKlsPdpNK4PRMtm',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [5]:
df_estados = spark.read.option("header", True).csv("exercicios/municipios-estados/csv/estados.csv")
print(df_estados.show(), df_estados.printSchema())

+---------+---+-------------------+--------+---------+--------+
|codigo_uf| uf|               nome|latitude|longitude|  regiao|
+---------+---+-------------------+--------+---------+--------+
|       11| RO|           Rondônia|  -10.83|   -63.34|   Norte|
|       12| AC|               Acre|   -8.77|   -70.55|   Norte|
|       13| AM|           Amazonas|   -3.47|    -65.1|   Norte|
|       14| RR|            Roraima|    1.99|   -61.33|   Norte|
|       15| PA|               Pará|   -3.79|   -52.48|   Norte|
|       16| AP|              Amapá|    1.41|   -51.77|   Norte|
|       17| TO|          Tocantins|   -9.46|   -48.26|   Norte|
|       21| MA|           Maranhão|   -5.42|   -45.44|Nordeste|
|       22| PI|              Piauí|    -6.6|   -42.28|Nordeste|
|       23| CE|              Ceará|    -5.2|   -39.53|Nordeste|
|       24| RN|Rio Grande do Norte|   -5.81|   -36.59|Nordeste|
|       25| PB|            Paraíba|   -7.28|   -36.72|Nordeste|
|       26| PE|         Pernambuco|   -8

In [6]:
df_municipios = spark.read.option("header", True).csv("exercicios/municipios-estados/csv/municipios.csv")
print(df_municipios.show(), df_municipios.printSchema())

+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|codigo_ibge|               nome|latitude|longitude|capital|codigo_uf|siafi_id|ddd|     fuso_horario|
+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|    5200050|    Abadia de Goiás|-16.7573| -49.4412|      0|       52|    1050| 62|America/Sao_Paulo|
|    3100104|Abadia dos Dourados|-18.4831| -47.3916|      0|       31|    4001| 34|America/Sao_Paulo|
|    5200100|          Abadiânia| -16.197| -48.7057|      0|       52|    9201| 62|America/Sao_Paulo|
|    3100203|             Abaeté|-19.1551| -45.4444|      0|       31|    4003| 37|America/Sao_Paulo|
|    1500107|         Abaetetuba|-1.72183| -48.8788|      0|       15|    0401| 91|America/Sao_Paulo|
|    2300101|            Abaiara|-7.34588| -39.0416|      0|       23|    1301| 88|America/Sao_Paulo|
|    2900108|             Abaíra|-13.2488| -41.6619|      0|       29|    3301| 77

In [17]:
df_1 = df_municipios.toPandas()
df_2 = df_estados.toPandas()

In [26]:
# juncao dos 2 datasets(estados e municipios)
df = df_1.merge(df_2[['uf', 'codigo_uf']], on='codigo_uf')
df

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo,GO
1,5200100,Abadiânia,-16.197,-48.7057,0,52,9201,62,America/Sao_Paulo,GO
2,5200134,Acreúna,-17.396,-50.3749,0,52,9645,64,America/Sao_Paulo,GO
3,5200159,Adelândia,-16.4127,-50.1657,0,52,9769,64,America/Sao_Paulo,GO
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,9771,62,America/Sao_Paulo,GO
...,...,...,...,...,...,...,...,...,...,...
5565,3306107,Valença,-22.2445,-43.7129,0,33,5921,24,America/Sao_Paulo,RJ
5566,3306156,Varre-Sai,-20.9276,-41.8701,0,33,2917,22,America/Sao_Paulo,RJ
5567,3306206,Vassouras,-22.4059,-43.6686,0,33,5923,24,America/Sao_Paulo,RJ
5568,3306305,Volta Redonda,-22.5202,-44.0996,0,33,5925,24,America/Sao_Paulo,RJ


In [36]:
csv_buffer = StringIO()
df_csv = df.to_csv(csv_buffer)
    
client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-08', Key=f'municipio_estado_spark_csv/cidades.csv')

{'ResponseMetadata': {'RequestId': '17C909B9962DFC58',
  'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx/1.19.2',
   'date': 'Tue, 23 Apr 2024 22:37:06 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"48eb67f2c42d8f653089ecd40ef4eeb5"',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'sa-east-1',
   'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'x-amz-request-id': '17C909B9962DFC58',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block'},
  'RetryAttempts': 0},
 'ETag': '"48eb67f2c42d8f653089ecd40ef4eeb5"'}

In [37]:
json_buffer = StringIO()
df_json = df.to_json(json_buffer)
    
client.put_object(Body=json_buffer.getvalue(), Bucket='aula-08', Key=f'v')

{'ResponseMetadata': {'RequestId': '17C909CC053B7D17',
  'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx/1.19.2',
   'date': 'Tue, 23 Apr 2024 22:38:25 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"9f9266e43d8fcb06dc29d1666c27b8bb"',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'sa-east-1',
   'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'x-amz-request-id': '17C909CC053B7D17',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block'},
  'RetryAttempts': 0},
 'ETag': '"9f9266e43d8fcb06dc29d1666c27b8bb"'}

In [44]:
df_spark = spark.createDataFrame(df)

In [45]:
df_spark.write.save("s3a://aula-08/municipio_estado_spark_parquet/cidades")